In [27]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [28]:
df = pd.read_csv("../Linear_regression_projet/data.csv")

In [29]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


## Traitement des données

In [30]:
df["date"] = pd.to_datetime(df["date"])
df["renovation"] = df["date"].dt.year - df['yr_built']
df['renovation'][df['yr_renovated'] != 0] = df["date"].dt.year - df["yr_renovated"]
df['renovation'][df['renovation'] < 0] = 0
df["date"] = (df["date"] - df["date"].min()).dt.days
df["ln_price"] = np.log(df["price"])
df["bathrooms"] -= 0.01
df["bath_or_shower"] = round(df["bathrooms"])
df["toilet"] = np.ceil(df["bathrooms"])
df = df[df["bedrooms"]<20] # 1 valeur abérante avec 30 chambres
df = df[df["price"] != 7700000] #Valeur dégradant le modèle

C:\Users\APPREN~1\AppData\Local\Temp/ipykernel_9336/785307995.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['renovation'][df['yr_renovated'] != 0] = df["date"].dt.year - df["yr_renovated"]
C:\Users\APPREN~1\AppData\Local\Temp/ipykernel_9336/785307995.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['renovation'][df['renovation'] < 0] = 0


## Standardisation des données

In [31]:
X = df.drop(["price", "ln_price", "id","yr_renovated","yr_built", "bathrooms"], axis=1)

In [32]:
X_scaled = StandardScaler().fit_transform(X)

## Modèle

In [33]:
pca = PCA(n_components=8)
projection = pca.fit_transform(X_scaled)

In [35]:
table = pd.DataFrame(projection, index=X.index, columns=["F"+str(i+1) for i in range(8)])
table

,F1,F2,F3,F4,F5,F6,F7,F8
0,-2.997040,-0.004041,-0.295161,0.793027,0.338120,-0.009974,0.235670,-0.006504
1,0.279509,-0.483912,-1.304028,0.924047,1.058046,-0.480643,0.256051,0.215692
2,-3.012737,0.672089,0.375104,0.340254,-0.007278,-0.528144,-2.180651,1.294627
3,-0.800839,-2.494225,0.567605,-1.627578,0.306737,-0.625031,1.316471,-1.165073
4,-0.433643,1.050496,-0.243629,0.008385,-0.163668,-0.805289,-0.883956,0.982273
...,...,...,...,...,...,...,...,...
21608,0.293699,0.446086,-2.577717,1.496414,0.927983,1.393811,1.013409,-0.328487
21609,0.842268,0.071533,-1.634692,0.927996,0.460303,-1.292174,0.840517,-1.006826
21610,-2.643421,0.901361,-1.604038,1.633095,0.339597,1.121091,0.870748,0.299308
21611,0.323621,1.762427,-1.210317,0.023240,-0.666474,-0.580589,0.061135,0.341679


## Détermination du nombre de components

On cherche à obtenir un seuil de 99%

In [ ]:
while pca.explained_variance_ratio.sum()<0.99:
    